In [14]:
from ollama import Client
client = Client(host='http://localhost:11434')
import yaml

# Load the yaml file as a global variable
with open('whoisthat/database.yml', 'r') as file:
  db = yaml.safe_load(file)
model = 'llama3'

In [15]:
def get_summary(book, bookmark, character):
  """
    Get a summary of the character's actions up to the bookmark in the book.
    This function uses the LLM to generate a summary from a famous book.
  """
  query = "I have read up to the end of " + bookmark
  query += " in the book '" +  book + "' by " +  db[book]['author']
  query += ". Describle what " +  character + " has done so far in 15 word or less."
  query += " Focus on key events and actions taken by this character."
  query += "Do not reveal spoilers for later sections of the book."
  response = client.chat(model=model, messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

def spoiler_check(book, character, summary):
  if book not in db or character not in db[book]['characters']:
    return "No spoilers in the database for " + character + " in " + book + "."
  query = "Read the following summary of " + character
  query += " from '" +  book + "' by " +  db[book]['author'] + ": '" + summary
  query += "'. Check to see whether the following spoiler is present: '"
  query += db[book]['characters'][character]['spoilers'][0] + "'. "
  query += "Give me a yes or no answer and a short one-sentence explanation for that decision."
  response = client.chat(model=model, messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

# Pride and Prejucice
1. Place a bookmark in your book
2. Generate a summary of the character of interest
3. Check for spoilers

In [16]:
book = 'Pride and Prejudice'
bookmark = 'Volume I' # e.g. Volume or Chapter
character = 'Mr. Wickham'
pride_and_prejudice_summary = get_summary(book, bookmark, character)
print(pride_and_prejudice_summary)

Mr. Wickham has been charming, flirtatious, and deceitful, eloping with Lydia Bennet without marrying her.


In [17]:
pride_and_prejudice_spolier_check = spoiler_check(book, character, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

Yes.

The summary includes the phrase "without marrying her", which implies that Mr. Wickham did not intend to marry Lydia Bennet despite eloping with her, indicating a breach of trust by Mr. Wickham.


# A Game of Thrones

In [18]:
book = 'A Game of Thrones'
bookmark = 'Chapter 1'
character = 'Jon Snow'
game_of_thrones_summary = get_summary(book, bookmark, character)
print(game_of_thrones_summary)

Jon Snow, a bastard, took his oath as a Night's Watch recruit at Castle Black.


In [19]:
print(spoiler_check(book, character, game_of_thrones_summary))

No.
This is because the provided summary only mentions Jon Snow's recruitment into the Night's Watch at Castle Black, but does not reveal his parentage or any connection to the Iron Throne. The information about Lyanna Stark, Rhaegar Targaryen, and Aegon Targaryen is not present in this summary.


In [20]:
tyrion_summary = get_summary(book, bookmark, 'Tyrion Lannister')
print(spoiler_check(book, 'Tyrion Lannister', tyrion_summary))

No spoilers in the database for Tyrion Lannister in A Game of Thrones.


# Harry Potter

In [21]:
book = 'Harry Potter and the Half-Blood Prince'
character = 'Albus Dumbledore'

chapter_20_summary = get_summary(book, 'Chapter 20', character)
print("Chapter 20 summary:")
print(chapter_20_summary)
print("")
print("")
print("Spoiler check for Chapter 20:")
print(spoiler_check(book, character, chapter_20_summary))
print("")
print("")

chapter_29_summary = get_summary(book, 'Chapter 29', character)
print("Chapter 29 summary:")
print(chapter_29_summary)
print("")
print("")
print("Spoiler check for Chapter 29:")
print(spoiler_check(book, character, chapter_29_summary))

Chapter 20 summary:
Dumbledore reveals Snape's past, helps Harry understand his destiny, and guides him through struggles at Hogwarts.


Spoiler check for Chapter 20:
No.

The reason is that according to the summary, Dumbledore "reveals Snape's past", which implies that Dumbledore is still alive at this point in the story.


Chapter 29 summary:
Dumbledore has appeared, guided Harry, revealed truth about Hogwarts founders' intentions, and maintained secrecy.


Spoiler check for Chapter 29:
Yes.

The summary mentions that Dumbledore has "maintained secrecy", which suggests that he is alive at some point in the story, as he would not be able to maintain secrecy if he were dead.
